Для запуска этого ноутбука необходим также chromewebdriver (.exe в случае с виндовс), который используется при работе библиотеки selenium. Скачать в инете под установленную версию хром. (chromewebdriver должен лежать в этой же папке)

Сам по себе этот ноутбук парсит картинки с сайта. При выполнении этой задачи возникло несколько проблем. 
1. Сайт написан на php и с него нельзя "в тупую" скачать картинки, потому что ссылка лежащая в src в теге img просто ведет на ту же страницу, которая и так открыта селениумом в настоящий момент.
2. Разрешение картинок 2560x1440. Скачивать такие картинки было бы не целесообразно, тк столь высокое разрешение
	- занимает много памяти на пк
	- значительно увеличивает время обработки фотографий
3. На сайте находится порядка 3000 картинок, перемещение между которыми можно легко осуществить заменой числа (id картинки) в url. На некоторых индексах картинки отсутствовали.

В итоге я пришел к костыльному решению, которое решает все эти проблемы. 
(Надо отметить, что если картинка по ссылке была, то ее разрешение было 4:3, если не было, то 1:1. Других разрешений не было вообще, но это стало понятно только после полного скачивания всех картинок с сайта)

Скачивание изображений производилось в три "группы". Первая - id от 0 до 100, Вторая - id от 101 до 1000, Третья - id от 1001 до 3177 (последний индекс на сайте). За это отвечают переменные start и end (индексы включены с обеих сторон).

### Решение
1. зайти на сайт
2. в цикле перебирать индексы
3. на каждой итерации находить картинку на сайте и проверять соотношение ее сторон
4. если 4:3 то картинка на сайте есть
    - Изменить окно браузера, чтобы вокруг картинки на сайте небыло рамок (оптимальный размер окна подбирался вручную)
    - Сделать скриншот окна и сохранить
5. если разрешение не 4:3 то картинки на сайте нет
    - Вывести в консоль "не 4 на 3, {отношение сторон}, {ширина}*{высота}, пропускаем id"
    - Сохранить id (на всякий случай)


In [1]:
# если папки images нет то ее нужно создать
import os

current_dir = os.getcwd()

if not os.path.exists(current_dir+'\\images'):
     os.mkdir(current_dir+'\\images')

In [2]:
from selenium import webdriver
import time
from tqdm import tqdm_notebook as tqdm
from url import main_url

main_url2 = 'http://rds.dermatology.ru/mephi/index.php?get={}'

options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")

options.add_experimental_option("excludeSwitches", ['enable-automation'])

driver = webdriver.Chrome(executable_path='chromedriver.exe', options=options)

bad_pictures_list = {}

start = 3155
end = 3155
try:
    driver.get(main_url)
    for picture_number in tqdm(range(start, end+1)):
        try:
            driver.get(main_url2.format(picture_number))
            driver.set_window_size(1920, 1080)

            image = driver.find_element_by_tag_name('img')
            width = int(image.get_attribute('width'))
            height = int(image.get_attribute('height'))

            if round(width/height, 1) == 1.3:
                driver.set_window_size(1016, 838)
                driver.save_screenshot(f"images/{picture_number}.png")

            else:
                print(f'не 4 на 3, {width/height}, {width}*{height}, пропускаем {picture_number}')
                bad_pictures_list[picture_number] = f'{width}*{height}'
                
                
        except Exception as ex:
            print(ex, 'ОШИБКА', picture_number)

except Exception as ex:
    print(ex)
finally:
    driver.close()
    driver.quit()

<ipython-input-2-cbe88d4dafc3>:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for picture_number in tqdm(range(start, end+1)):


  0%|          | 0/1 [00:00<?, ?it/s]